In [56]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

client_id = 'client_660b4d0cceb4cf9f989acef154caf4ab'
client_secret = 'secret_bd749d4e48d050deb1d8873b061456f2'
#client_id = 'client_ccde1b02ebfe494290bc6156fd6e8e98'
#client_secret = 'key_822f032d940be843b7493614f01206f3'

def retrive_properties(client_id, client_secret, pageNumber, postCode="", state="VIC", pageSize=200, filename="propertiesInfo.csv"):
  scopes = ['api_listings_read']
  auth_url = 'https://auth.domain.com.au/v1/connect/token'
  url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

  response = requests.post(auth_url, data = {
                          'client_id':client_id,
                          'client_secret':client_secret,
                          'grant_type':'client_credentials',
                          'scope':scopes,
                          'Content-Type':'text/json'
                          })
  json_res = response.json()
  access_token=json_res['access_token']
  print("Access:", access_token)
  auth = {'Authorization':'Bearer ' + access_token}
  url = url_endpoint

  post_fields = {
        "listingType":"Rent",
        "pageSize": pageSize, 
        "pageNumber": pageNumber, # 10 per page
        "propertyTypes":"",
        "minBedrooms":1,
        "minBathrooms":1,
        # "maxPrice": 400,
        "sort": {
            "sortKey": "Default"
        },

        "locations":[
          {
            "state":state,
            "region":"",
            "area":"",
            "suburb":"",
            "postCode":postCode,
            "includeSurroundingSuburbs":False
          }
        ]
  }

  # Request content
  respond = requests.post(url,headers=auth,json=post_fields)
  content = json.loads(respond.text)
  print("Loaded data: " + str(len(content)))
  print(content)
  if content["type"] != "PropertyListing" or "errors" in content or len(content) == 0:
    print("Error accurs")
    print(content)
    return False

  # Get request for each property ID and extract attributes
  for i in content:
      #get details
      id = i['listing']['id']
      listing_type = i['listing']['listingType']
      r = i['listing']['propertyDetails']
      street_address=r['displayableAddress']
      suburb=r['suburb']
      postcode=r['postcode']
      area=r['area']
      latitude = r['latitude']
      longitude = r['longitude']
      property_type=r['propertyType']
      bathrooms=r['bathrooms']
      bedrooms=r['bedrooms']

      if(str(i).find('carspaces'))>0:
          carspaces=i['listing']['propertyDetails']['carspaces']
      else:
          carspaces=0

      price=i['listing']['priceDetails']['displayPrice'] #[1:4]

      info = [id, listing_type, price, property_type, area, bedrooms, bathrooms, carspaces, street_address, suburb, postcode, latitude, longitude]

      with open(filename, 'a', newline='') as g:
          thewriter = writer(g)
          thewriter.writerow(info)

  return True

# retrive_properties(client_id, client_secret, 1)

Rrtrive rental data

In [55]:

# Retrive rental data by postcode
# postcode 3000~4000
for i in range(3000, 4000):
    print("Retriving data from postcode: " + str(i))
    for p in range(1, 6):
        if not retrive_properties(client_id, client_secret, p, str(i)):
            break

# postcode 8000~9000
for i in range(8000, 9000):
    print("Retriving data from postcode: " + str(i))
    for p in range(1, 6):
        if not retrive_properties(client_id, client_secret, p, str(i)):
            break

Retriving data from postcode: 3000


KeyError: 'access_token'